### Fine-tune BERT for classification

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load CSV files
train_df = pd.read_csv('data/collections/train.txt', sep='\t')
test_df = pd.read_csv('data/collections/test.txt', sep='\t')


c:\Users\plush\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label_map = {'positive': 0, 'neutral': 1, 'negative': 2}
train_df['label'] = train_df['sentiment'].map(label_map)
test_df['label'] = test_df['sentiment'].map(label_map)

In [3]:
train_df.head()

,id,sentiment,tweet,label
0,638165350966669312,negative,Nicki's butt is just too big like c'mon that's...,2
1,640169120600862720,neutral,Haruna Lukmon may av just played himself out o...,1
2,635946254254624769,neutral,Zach Putnam will be unavailable for the White ...,1
3,667121920333258752,negative,"""""""@daithimckay what about the victims of IRA ...",2
4,628637519643586560,neutral,"""""""LHP Matt Boyd, traded to @tigers in David P...",1


In [4]:
test_dataset = Dataset.from_pandas(test_df[['tweet', 'label']])

train_dataset, val_dataset = train_test_split(train_df, test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_dataset[['tweet', 'label']])
val_dataset = Dataset.from_pandas(val_dataset[['tweet', 'label']])

In [5]:
from transformers import DistilBertTokenizer

# Load the pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True, max_length=128)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 4662/4662 [00:01<00:00, 3235.61 examples/s]


In [6]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load the pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import torch

# Check if GPU is available and move model to GPU if so
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [8]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # Increase batch size
    per_device_eval_batch_size=64,   # Larger evaluation batch size
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps if batch size is too large
)

# Define the Trainer
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    tokenizer=tokenizer,                 # tokenizer to handle the tokenization
)

c:\Users\plush\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\plush\AppData\Local\Temp\ipykernel_31160\2661823310.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()

  1%|▏         | 10/786 [01:23<1:47:59,  8.35s/it]

{'loss': 1.0591, 'grad_norm': 2.382056951522827, 'learning_rate': 1.974554707379135e-05, 'epoch': 0.04}


  3%|▎         | 20/786 [02:50<1:50:01,  8.62s/it]

{'loss': 1.0353, 'grad_norm': 1.9395687580108643, 'learning_rate': 1.9491094147582698e-05, 'epoch': 0.08}


  4%|▍         | 30/786 [04:15<1:45:50,  8.40s/it]

...

 99%|█████████▉| 780/786 [1:48:59<00:48,  8.01s/it]

{'loss': 0.4997, 'grad_norm': 8.768998146057129, 'learning_rate': 1.5267175572519085e-07, 'epoch': 2.97}


                                                   
100%|██████████| 786/786 [1:50:58<00:00,  8.47s/it]

{'eval_loss': 0.6499510407447815, 'eval_runtime': 69.2581, 'eval_samples_per_second': 26.928, 'eval_steps_per_second': 0.433, 'epoch': 2.99}
{'train_runtime': 6658.3078, 'train_samples_per_second': 7.561, 'train_steps_per_second': 0.118, 'train_loss': 0.6156937730221348, 'epoch': 2.99}


TrainOutput(global_step=786, training_loss=0.6156937730221348, metrics={'train_runtime': 6658.3078, 'train_samples_per_second': 7.561, 'train_steps_per_second': 0.118, 'total_flos': 1664681251908096.0, 'train_loss': 0.6156937730221348, 'epoch': 2.994285714285714})

In [10]:
eval_results = trainer.evaluate()
print(eval_results)

100%|██████████| 30/30 [01:07<00:00,  2.26s/it]

{'eval_loss': 0.6499510407447815, 'eval_runtime': 70.1647, 'eval_samples_per_second': 26.58, 'eval_steps_per_second': 0.428, 'epoch': 2.994285714285714}


In [11]:
# Save the model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.txt',
 './fine_tuned_model\\added_tokens.json')

### Run test on transformer

In [15]:
# Load the fine-tuned model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_model")
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_model")

# Example inference
input_text = "Example positive tweet. Very good comments"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Get predictions
outputs = model(**inputs)
logits = outputs.logits
prediction = logits.argmax(dim=-1).item()
print(f"Predicted class: {prediction}")


Predicted class: 0
